In [ ]:
#Ongoing list of imports/packages
import scipy as sp
from pylab import *
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colormaps
import networkx as nx
import seaborn as sns
import collections
from IPython.display import clear_output
import random
from scipy.stats import mannwhitneyu
import pandas as pd
from google.colab import drive
from scipy import stats
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster, cophenet
from scipy.spatial.distance import pdist
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import adjusted_rand_score
from sklearn.tree import plot_tree
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import accuracy_score
from scipy.stats import ttest_ind
from scipy.stats import shapiro
import datetime
!pip install fastcluster
import fastcluster

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Data cleaning and reorganization from GEO accession
### Work flow and pseudocode is below

Following the GEO accession there are two files. One file we are assuming comes from the manufacturer of the RNA microarray or as a read out has an index, NM ID, common gene abbreviation, full name of gene, GO accession and probe sequences. The second file has indexes and result values by patient. This also includes metadata on the patients.

After uploading the .txt files. Metadata was removed and the files were merged so the final dataset has the corresponding NM IDs, index value, gene abb, and patient samples expression values.

The next file contains all of the phenotypic data from the samples (there can be multiple samples from a patient)


In [ ]:

#uploading GEO accesion data
file_path = '/content/drive/MyDrive/20440_Project/GSE11223_series_matrix.txt'
#converting txt to data frame (removing metadata)
raw_file = pd.read_csv(file_path, delimiter='\t', skiprows=50, header=0)

#uploading Microarray GeneID data probes and
file_path = '/content/drive/MyDrive/20440_Project/GeneID_Probes.txt'
#converting txt to data frame
ID_raw = pd.read_csv(file_path, delimiter='\t', header = None)


NotADirectoryError: [Errno 20] Not a directory: '/content/drive/MyDrive/20440_Project/GSE11223_series_matrix.txt'

In [ ]:
#extracting ID_ref, NM_... ID, and Gene Names
ID_abb = ID_raw.iloc[:,[0,3,9]]
Column_names = ['ID_REF', 'NM_ID', 'GeneID']
ID_abb.columns = Column_names
#making sure everything is same dtype before merge
ID_abb.loc[:, 'ID_REF']=ID_abb.loc[:, "ID_REF"].astype(str)
ID_abb.loc[:, 'NM_ID']=ID_abb.loc[:, "NM_ID"].astype(str)
ID_abb.loc[:, 'GeneID']=ID_abb.loc[:, "GeneID"].astype(str)
#Replacing GeneID with NM_ID if blank
for index, row in ID_abb.iterrows():
  if row['GeneID']== 'nan':
    ID_abb.at[index, 'GeneID'] = row['NM_ID']

#Renaming ID_REF column on raw data
column = list(raw_file.columns)
raw_file.rename(columns= {column[0]: 'ID_REF'}, inplace = True)
#making sure everything is same dtype before merge
raw_file.loc[:, 'ID_REF'] = raw_file.loc[:, 'ID_REF'].astype(str)
#replacing NaN values with 0
raw_file = raw_file.fillna(0)

#adding NM_... ID and Gene Names to patient values
merged_df = pd.merge(raw_file, ID_abb, on='ID_REF', how ='left')

#extracting the name of the columns
column_names = list(merged_df.columns)

#adding NM_ID, GeneID to front of df
new_order = column_names[-2:] + column_names[:-2]
merged_df_reorg = merged_df[new_order]

In [ ]:
#uploading sample metadata
file_path = '/content/drive/MyDrive/20440_Project/GSE11223_newPheno.txt'
#converting txt to data frame
sample_info = pd.read_csv(file_path, delimiter='\t')
print(sample_info.info())

## Data manipulation
### According to Nobel et al the data has already been normalized and therefore we can go straight into the clustering procedure. We are going to perform heirarchical clustering using a variety of parameters and evaluators.  

In [ ]:
#Transposing data so each sample is a row and each gene expression value is a column
merged_df_trans = merged_df_reorg.transpose()
Sample_type_ex = merged_df_trans.iloc[3:, 89:-1]
print(Sample_type_ex.head())

In [ ]:
#Generating heatmap
heatmap = sns.clustermap(Sample_type_ex.apply(pd.to_numeric, errors='coerce'), cmap='viridis', standard_scale=1)
plt.title('Heatmap and Clustering of Normalized Expression Data and Sample IDs')
heatmap.ax_heatmap.set_xlabel("GENE Expression Number")
heatmap.ax_heatmap.set_ylabel("Sample ID Number")

output_local = '/content/drive/MyDrive/20440_Project/Figures/'
plt.savefig(output_local + 'heatmap.jpeg', format = 'jpeg' , bbox_inches='tight')


### Determining the accuracy of the clustering with different n-clusters
(Normal vs UC n = 2)

(Normal vs inflammed n = 2)

(Biopsy type n = 4)

(Disease and biopsy type n = 14)

##Calculating the silhouette coefficient, calinski harabasz score, and the davies bouldin score for each scenerio

In [ ]:
Sample_type_ex_numeric = Sample_type_ex.apply(pd.to_numeric, errors='coerce').dropna()


In [ ]:
#Clustering by disease + inflammation + location n = 14
Sample_type_ex_numeric.columns = Sample_type_ex_numeric.columns.astype(str)
kmeansA = KMeans(n_clusters=14)
cluster_labelsA = kmeansA.fit_predict(Sample_type_ex_numeric)

# Calculate silhouette score
silhouette_avgA = silhouette_score(Sample_type_ex_numeric, cluster_labelsA)
print("Silhouette Score (n=14):", silhouette_avgA)

# Calculate Calinski-Harabasz Index
calinski_harabasz_score_valueA = calinski_harabasz_score(Sample_type_ex_numeric, cluster_labelsA)
print("Calinski-Harabasz Index (n=14):", calinski_harabasz_score_valueA)

# Calculate Davies-Bouldin Index
davies_bouldin_score_valueA = davies_bouldin_score(Sample_type_ex_numeric, cluster_labelsA)
print("Davies-Bouldin Index (n=14):", davies_bouldin_score_valueA)

# Add cluster labels to your dataframe
Sample_type_ex['Cluster_Labels_DIL'] = cluster_labelsA


#Clustering by disease n = 2
Sample_type_ex_numeric.columns = Sample_type_ex_numeric.columns.astype(str)
kmeansB = KMeans(n_clusters=2)
cluster_labelsB = kmeansB.fit_predict(Sample_type_ex_numeric)

# Calculate silhouette score
silhouette_avgB = silhouette_score(Sample_type_ex_numeric, cluster_labelsB)
print("Silhouette Score (n=2):", silhouette_avgB)

# Calculate Calinski-Harabasz Index
calinski_harabasz_score_valueB = calinski_harabasz_score(Sample_type_ex_numeric, cluster_labelsB)
print("Calinski-Harabasz Index (n=2):", calinski_harabasz_score_valueB)

# Calculate Davies-Bouldin Index
davies_bouldin_score_valueB = davies_bouldin_score(Sample_type_ex_numeric, cluster_labelsB)
print("Davies-Bouldin Index (n=2):", davies_bouldin_score_valueB)

# Add cluster labels to your dataframe
Sample_type_ex['Cluster_Labels_D'] = cluster_labelsB


#Clustering by location n= 4
Sample_type_ex_numeric.columns = Sample_type_ex_numeric.columns.astype(str)
kmeansC = KMeans(n_clusters=4)
cluster_labelsC = kmeansC.fit_predict(Sample_type_ex_numeric)

# Calculate silhouette score
silhouette_avgC = silhouette_score(Sample_type_ex_numeric, cluster_labelsC)
print("Silhouette Score (n=4):", silhouette_avgC)

# Calculate Calinski-Harabasz Index
calinski_harabasz_score_valueC = calinski_harabasz_score(Sample_type_ex_numeric, cluster_labelsC)
print("Calinski-Harabasz Index (n=4):", calinski_harabasz_score_valueC)

# Calculate Davies-Bouldin Index
davies_bouldin_score_valueC = davies_bouldin_score(Sample_type_ex_numeric, cluster_labelsC)
print("Davies-Bouldin Index (n=4):", davies_bouldin_score_valueC)

# Add cluster labels to your dataframe
Sample_type_ex['Cluster_Labels_L'] = cluster_labelsC



##Calculating the ARI and accuracy of the clustering

In [ ]:
#Rearranging and cleaning the data for analysis

sample_cluster_df = Sample_type_ex.iloc[:, [-3, -2, -1]]

# Convert index to a separate column
sample_cluster_df['Index_Column'] = sample_cluster_df.index

# Split the index column by spaces and expand it into separate columns
split_columns = sample_cluster_df['Index_Column'].str.split(expand=True)

# Assign the split columns to the DataFrame
sample_cluster_df2 = pd.concat([sample_cluster_df, split_columns], axis=1)

# Drop the temporary column if needed
sample_cluster_df2.drop(columns=['Index_Column'], inplace=True)

# Convert the last column to a separate column
sample_cluster_df2['Last_Column'] = sample_cluster_df2.iloc[:, -1]

# Split the last column by periods and expand it into separate columns
split_columns = sample_cluster_df2['Last_Column'].str.split('.', expand=True)

# Assign the split columns to the DataFrame
sample_cluster_df3 = pd.concat([sample_cluster_df2, split_columns], axis=1)

# Drop the temporary column if needed
sample_cluster_df3.drop(columns=['Last_Column'], inplace=True)

print(sample_cluster_df3)

In [ ]:
#Setting index to UC or Normal
sample_cluster_df4 = sample_cluster_df3.set_index(sample_cluster_df3.columns[3])

# Extracting "Normal" or "UC" from the index and setting it as the first column
sample_cluster_df3['Biopsy_Type'] = sample_cluster_df3.index.str[0]

# Reordering the columns
sample_cluster_df3 = sample_cluster_df3[['Biopsy_Type'] + [col for col in sample_cluster_df3.columns if col != 'Biopsy_Type']]

# Resetting the index
sample_cluster_df3.reset_index(drop=True, inplace=True)
sample_cluster_df4 = sample_cluster_df3.iloc[:, [0,2]]

# Replace 'Normal' with 0 and 'UC' with 1 in the 'Biopsy_Type' column
sample_cluster_df4['Biopsy_Type'] = sample_cluster_df4['Biopsy_Type'].replace({'N': 0, 'U': 1})

# Get the ground truth labels
ground_truth_labels = sample_cluster_df4['Biopsy_Type'].values

# Assuming 'ground_truth_labels' are the true labels and 'cluster_labelsB' are the labels obtained from clustering
ari = adjusted_rand_score(ground_truth_labels, cluster_labelsB)

# Print the ARI score
print("Adjusted Rand Index (ARI):", ari)



# Assuming 'ground_truth_labels' are the true labels and 'cluster_labels' are the labels obtained from clustering
accuracy = accuracy_score(ground_truth_labels, cluster_labelsB)

# Print the accuracy score
print("Accuracy:", accuracy)


## Differentially expressed genes between UC and Healthy Patients

In [ ]:
##WORKS
# Determine the differentially expressed genes between healthy and UC
merged_df_trans2 = merged_df_trans.iloc[:, 89:-1]
merged_df_trans2.index = merged_df_trans2.index.str.split().str[0]

# Define the index names for healthy and disease samples
Healthy = 'Normal'
Disease = 'UC'

# Selecting rows
Healthy_rows = merged_df_trans2[merged_df_trans2.index == Healthy]
UC_rows = merged_df_trans2[merged_df_trans2.index == Disease]

# Initialize an empty DataFrame to store p-values
p_values_df = pd.DataFrame(columns=merged_df_trans2.columns)
fold_change_df = pd.DataFrame(columns = merged_df_trans2.columns )

# Calculate average fold change (FC) for each gene
for column in merged_df_trans2.columns:
    # Convert data to numeric to handle missing values
    Healthy_column_numeric = pd.to_numeric(Healthy_rows[column], errors='coerce')
    UC_column_numeric = pd.to_numeric(UC_rows[column], errors='coerce')

    # Drop missing values
    Healthy_column_numeric = Healthy_column_numeric.dropna()
    UC_column_numeric = UC_column_numeric.dropna()

    # Calculate mean expression in healthy and disease samples
    mean_expression_healthy = Healthy_column_numeric.mean()
    mean_expression_UC = UC_column_numeric.mean()

    # Calculate fold change
    fold_change = mean_expression_UC / mean_expression_healthy
    fold_change_df[column] = [fold_change]

    # Perform t-test if both healthy and disease samples exist
    if not (Healthy_column_numeric.empty or UC_column_numeric.empty):
        t_stat, p_val = ttest_ind(UC_column_numeric, Healthy_column_numeric)
        # Add p-value for each gene to the DataFrame
        p_values_df[column] = [p_val]

# Set index names for fold change and p-values dataframes
fold_change_df.index = ['Fold change']
p_values_df.index = ['p-val']

# Concatenate the DataFrame containing FC and p-values to the original DataFrame
EX_FC_pvals = pd.concat([merged_df_trans2, fold_change_df, p_values_df], ignore_index = False)



In [ ]:
#WORKS
#Transforming all gene p-vals and FC
p_vals = EX_FC_pvals.iloc[-1,:]
Fold_changes = EX_FC_pvals.iloc[-2,:]

# Convert p-values to numeric (in case they are not already)
p_vals = pd.to_numeric(p_vals, errors='coerce')
# Take the logarithm of p-values (base 10) for better visualization
log10pval_all = -np.log10(p_vals)

# Convert p-values to numeric (in case they are not already)
Fold_changes = pd.to_numeric(Fold_changes, errors='coerce')
# Take the logarithm of p-values (base 10) for better visualization
log2FC_all = -np.log2(Fold_changes)

# Plotting volcano plot

VolcanoPlot = plt.scatter(log2FC_all, log10pval_all, alpha = 0.5, color = 'blue', label = 'all' )
plt.xlabel("log2(Fold Change) [UC/Healthy]")
plt.ylabel("-log(p-value)")
plt.title("Volcano plot differentially expressed genes between UC and Healthy ")
plt.axhline(-np.log10(0.05), color='black', linestyle='--', linewidth=1)  # Add a horizontal line for significance threshold
plt.axvline(3, color='green', linestyle='--', linewidth=1)  # Add a vertical line for fold change threshold
plt.axvline(-3, color='green', linestyle='--', linewidth=1)  # Add a vertical line for fold change threshold
output_local = '/content/drive/MyDrive/20440_Project/Figures/'
plt.savefig(output_local + 'VolcanoPlot_All', format = 'jpeg' , bbox_inches='tight')

# Filter only columns with p-values less than 0.05
significant_genes = EX_FC_pvals.loc[:, EX_FC_pvals.iloc[-1] < 0.05]

# Filter only columns with absolute FC values exceeding 1
significant_genes2 = significant_genes.loc[:, (abs(significant_genes.iloc[-2]) > 3)]

# Iterate over columns in the DataFrame
for column in significant_genes2.columns:
    # Check if the absolute FC value is within the desired range
    if abs(significant_genes2[column].iloc[-2]) <= 3:
        # Drop the column if the condition is not met
        significant_genes2.drop(column, axis=1, inplace=True)

GOI = significant_genes2.iloc[[1,205,206],:]

In [ ]:
GOI2 = GOI.transpose()
print("Total number of Genes of Interest is",len(GOI2))
GOI2_sort = GOI2.sort_values(by=GOI2.columns[1], ascending = False)
#Saving output as .csv to figure folder
GOI2_sort.to_csv('/content/drive/MyDrive/20440_Project/Figures/Genes_of_interest.csv', index = False)
#Saving output as a .txt to figure folder
GOI2_sort.to_csv('/content/drive/MyDrive/20440_Project/Figures/Genes_of_interest.txt', sep='\t', index = False)
print(GOI2_sort.iloc[0:5,:])
print(GOI2_sort.iloc[-5:,:])

#Performing differential gene expression by biopsy location

In [ ]:
# Remove the '.number' suffix from the index
merged_df_trans = merged_df_reorg.transpose()
Sample_type_ex = merged_df_trans.iloc[3:, 89:-1]
Sample_type_ex.index = Sample_type_ex.index.str.replace(r'\.\d+$', '', regex=True)
split = Sample_type_ex.index.str.split()
first_word = split.str[0]
second_word = split.str[1]
third_word = split.str[2]
fourth_word = split.str[3]
fourth = fourth_word.str.split('.')
Sample_type_ex['Condition'] = first_word
Sample_type_ex['Biopsy'] = third_word + fourth_word

In [ ]:
Sample_type_ex2 = Sample_type_ex.set_index('Condition')
# Select all rows where the 'Biopsy' column is 'sigmoidcolon'
sigmoid_colon_dataA = Sample_type_ex2[Sample_type_ex2['Biopsy'] == 'sigmoidcolon']
descending_colon_dataA = Sample_type_ex2[Sample_type_ex2['Biopsy'] == 'descendingcolon']
terminal_ileum_dataA = Sample_type_ex2[Sample_type_ex2['Biopsy'] == 'terminalileum']
ascending_colon_dataA = Sample_type_ex2[Sample_type_ex2['Biopsy'] == 'ascendingcolon']

In [ ]:
print(len(sigmoid_colon_dataA))
print(len(descending_colon_dataA))
print(len(terminal_ileum_dataA))
print(len(ascending_colon_dataA))

In [ ]:
#remove the last coulumn
sigmoid_colon_data = sigmoid_colon_dataA.iloc[:,:-1]
descending_colon_data = descending_colon_dataA.iloc[:,:-1]
terminal_ileum_data = terminal_ileum_dataA.iloc[:,:-1]
ascending_colon_data = ascending_colon_dataA.iloc[:,:-1]

In [ ]:
# Define the index names for healthy and disease samples
Healthy = 'Normal'
Disease = 'UC'

# Selecting rows
Healthy_rows = sigmoid_colon_data[sigmoid_colon_data.index == Healthy]
UC_rows = sigmoid_colon_data[sigmoid_colon_data.index == Disease]

# Initialize an empty DataFrame to store p-values
p_values_df = pd.DataFrame(columns=sigmoid_colon_data.columns)
fold_change_df = pd.DataFrame(columns = sigmoid_colon_data.columns )

# Calculate average fold change (FC) for each gene
for column in sigmoid_colon_data.columns:
    # Convert data to numeric to handle missing values
    Healthy_column_numeric = pd.to_numeric(Healthy_rows[column], errors='coerce')
    UC_column_numeric = pd.to_numeric(UC_rows[column], errors='coerce')

    # Drop missing values
    Healthy_column_numeric = Healthy_column_numeric.dropna()
    UC_column_numeric = UC_column_numeric.dropna()

    # Calculate mean expression in healthy and disease samples
    mean_expression_healthy = Healthy_column_numeric.mean()
    mean_expression_UC = UC_column_numeric.mean()

    # Calculate fold change
    fold_change = mean_expression_UC / mean_expression_healthy
    fold_change_df[column] = [fold_change]

    # Perform t-test if both healthy and disease samples exist
    if not (Healthy_column_numeric.empty or UC_column_numeric.empty):
        t_stat, p_val = ttest_ind(UC_column_numeric, Healthy_column_numeric)
        # Add p-value for each gene to the DataFrame
        p_values_df[column] = [p_val]

# Set index names for fold change and p-values dataframes
fold_change_df.index = ['Fold change']
p_values_df.index = ['p-val']

# Concatenate the DataFrame containing FC and p-values to the original DataFrame
EX_FC_pvals = pd.concat([sigmoid_colon_data, fold_change_df, p_values_df], ignore_index = False)

#WORKS
#Transforming all gene p-vals and FC
p_vals = EX_FC_pvals.iloc[-1,:]
Fold_changes = EX_FC_pvals.iloc[-2,:]

# Convert p-values to numeric (in case they are not already)
p_vals = pd.to_numeric(p_vals, errors='coerce')
# Take the logarithm of p-values (base 10) for better visualization
log10pval_all = -np.log10(p_vals)

# Convert p-values to numeric (in case they are not already)
Fold_changes = pd.to_numeric(Fold_changes, errors='coerce')
# Take the logarithm of p-values (base 10) for better visualization
log2FC_all = -np.log2(Fold_changes)

# Plotting volcano plot

VolcanoPlot = plt.scatter(log2FC_all, log10pval_all, alpha = 0.5, color = 'blue', label = 'all' )
plt.xlabel("log2(Fold Change) [UC/Healthy]")
plt.ylabel("-log(p-value)")
plt.title("Volcano plot differentially expressed genes between UC and Healthy in sigmoid colon")
plt.axhline(-np.log10(0.05), color='black', linestyle='--', linewidth=1)  # Add a horizontal line for significance threshold
plt.axvline(3, color='green', linestyle='--', linewidth=1)  # Add a vertical line for fold change threshold
plt.axvline(-3, color='green', linestyle='--', linewidth=1)  # Add a vertical line for fold change threshold
output_local = '/content/drive/MyDrive/20440_Project/Figures/'
plt.savefig(output_local + 'VolcanoPlot_Sigmoid_Colon', format = 'jpeg' , bbox_inches='tight')

# Filter only columns with p-values less than 0.05
significant_genes = EX_FC_pvals.loc[:, EX_FC_pvals.iloc[-1] < 0.05]

# Filter only columns with absolute FC values exceeding 3
significant_genes2 = significant_genes.loc[:, (abs(significant_genes.iloc[-2]) > 3)]

# Iterate over columns in the DataFrame
for column in significant_genes2.columns:
    # Check if the absolute FC value is within the desired range
    if abs(significant_genes2[column].iloc[-2]) <= 3:
        # Drop the column if the condition is not met
        significant_genes2.drop(column, axis=1, inplace=True)

GOI_sigmoid_colon = significant_genes2


In [ ]:
# Define the index names for healthy and disease samples
Healthy = 'Normal'
Disease = 'UC'

# Selecting rows
Healthy_rows = descending_colon_data[descending_colon_data.index == Healthy]
UC_rows = descending_colon_data[descending_colon_data.index == Disease]

# Initialize an empty DataFrame to store p-values
p_values_df = pd.DataFrame(columns=descending_colon_data.columns)
fold_change_df = pd.DataFrame(columns = descending_colon_data.columns )

# Calculate average fold change (FC) for each gene
for column in descending_colon_data.columns:
    # Convert data to numeric to handle missing values
    Healthy_column_numeric = pd.to_numeric(Healthy_rows[column], errors='coerce')
    UC_column_numeric = pd.to_numeric(UC_rows[column], errors='coerce')

    # Drop missing values
    Healthy_column_numeric = Healthy_column_numeric.dropna()
    UC_column_numeric = UC_column_numeric.dropna()

    # Calculate mean expression in healthy and disease samples
    mean_expression_healthy = Healthy_column_numeric.mean()
    mean_expression_UC = UC_column_numeric.mean()

    # Calculate fold change
    fold_change = mean_expression_UC / mean_expression_healthy
    fold_change_df[column] = [fold_change]

    # Perform t-test if both healthy and disease samples exist
    if not (Healthy_column_numeric.empty or UC_column_numeric.empty):
        t_stat, p_val = ttest_ind(UC_column_numeric, Healthy_column_numeric)
        # Add p-value for each gene to the DataFrame
        p_values_df[column] = [p_val]

# Set index names for fold change and p-values dataframes
fold_change_df.index = ['Fold change']
p_values_df.index = ['p-val']

# Concatenate the DataFrame containing FC and p-values to the original DataFrame
EX_FC_pvals = pd.concat([descending_colon_data, fold_change_df, p_values_df], ignore_index = False)

#WORKS
#Transforming all gene p-vals and FC
p_vals = EX_FC_pvals.iloc[-1,:]
Fold_changes = EX_FC_pvals.iloc[-2,:]

# Convert p-values to numeric (in case they are not already)
p_vals = pd.to_numeric(p_vals, errors='coerce')
# Take the logarithm of p-values (base 10) for better visualization
log10pval_all = -np.log10(p_vals)

# Convert p-values to numeric (in case they are not already)
Fold_changes = pd.to_numeric(Fold_changes, errors='coerce')
# Take the logarithm of p-values (base 10) for better visualization
log2FC_all = -np.log2(Fold_changes)

# Plotting volcano plot

VolcanoPlot = plt.scatter(log2FC_all, log10pval_all, alpha = 0.5, color = 'blue', label = 'all' )
plt.xlabel("log2(Fold Change) [UC/Healthy]")
plt.ylabel("-log(p-value)")
plt.title("Volcano plot differentially expressed genes between UC and Healthy in descending colon")
plt.axhline(-np.log10(0.05), color='black', linestyle='--', linewidth=1)  # Add a horizontal line for significance threshold
plt.axvline(3, color='green', linestyle='--', linewidth=1)  # Add a vertical line for fold change threshold
plt.axvline(-3, color='green', linestyle='--', linewidth=1)  # Add a vertical line for fold change threshold
output_local = '/content/drive/MyDrive/20440_Project/Figures/'
plt.savefig(output_local + 'VolcanoPlot_Descending_Colon', format = 'jpeg' , bbox_inches='tight')

# Filter only columns with p-values less than 0.05
significant_genes = EX_FC_pvals.loc[:, EX_FC_pvals.iloc[-1] < 0.05]

# Filter only columns with absolute FC values exceeding 3
significant_genes2 = significant_genes.loc[:, (abs(significant_genes.iloc[-2]) > 3)]

# Iterate over columns in the DataFrame
for column in significant_genes2.columns:
    # Check if the absolute FC value is within the desired range
    if abs(significant_genes2[column].iloc[-2]) <= 3:
        # Drop the column if the condition is not met
        significant_genes2.drop(column, axis=1, inplace=True)

GOI_descending_colon = significant_genes2


In [ ]:
# Define the index names for healthy and disease samples
Healthy = 'Normal'
Disease = 'UC'

# Selecting rows
Healthy_rows = terminal_ileum_data[terminal_ileum_data.index == Healthy]
UC_rows = terminal_ileum_data[terminal_ileum_data.index == Disease]

# Initialize an empty DataFrame to store p-values
p_values_df = pd.DataFrame(columns=terminal_ileum_data.columns)
fold_change_df = pd.DataFrame(columns = terminal_ileum_data.columns )

# Calculate average fold change (FC) for each gene
for column in terminal_ileum_data.columns:
    # Convert data to numeric to handle missing values
    Healthy_column_numeric = pd.to_numeric(Healthy_rows[column], errors='coerce')
    UC_column_numeric = pd.to_numeric(UC_rows[column], errors='coerce')

    # Drop missing values
    Healthy_column_numeric = Healthy_column_numeric.dropna()
    UC_column_numeric = UC_column_numeric.dropna()

    # Calculate mean expression in healthy and disease samples
    mean_expression_healthy = Healthy_column_numeric.mean()
    mean_expression_UC = UC_column_numeric.mean()

    # Calculate fold change
    fold_change = mean_expression_UC / mean_expression_healthy
    fold_change_df[column] = [fold_change]

    # Perform t-test if both healthy and disease samples exist
    if not (Healthy_column_numeric.empty or UC_column_numeric.empty):
        t_stat, p_val = ttest_ind(UC_column_numeric, Healthy_column_numeric)
        # Add p-value for each gene to the DataFrame
        p_values_df[column] = [p_val]

# Set index names for fold change and p-values dataframes
fold_change_df.index = ['Fold change']
p_values_df.index = ['p-val']

# Concatenate the DataFrame containing FC and p-values to the original DataFrame
EX_FC_pvals = pd.concat([terminal_ileum_data, fold_change_df, p_values_df], ignore_index = False)

#WORKS
#Transforming all gene p-vals and FC
p_vals = EX_FC_pvals.iloc[-1,:]
Fold_changes = EX_FC_pvals.iloc[-2,:]

# Convert p-values to numeric (in case they are not already)
p_vals = pd.to_numeric(p_vals, errors='coerce')
# Take the logarithm of p-values (base 10) for better visualization
log10pval_all = -np.log10(p_vals)

# Convert p-values to numeric (in case they are not already)
Fold_changes = pd.to_numeric(Fold_changes, errors='coerce')
# Take the logarithm of p-values (base 10) for better visualization
log2FC_all = -np.log2(Fold_changes)

# Plotting volcano plot

VolcanoPlot = plt.scatter(log2FC_all, log10pval_all, alpha = 0.5, color = 'blue', label = 'all' )
plt.xlabel("log2(Fold Change) [UC/Healthy]")
plt.ylabel("-log(p-value)")
plt.title("Volcano plot differentially expressed genes between UC and Healthy in terminal ileum")
plt.axhline(-np.log10(0.05), color='black', linestyle='--', linewidth=1)  # Add a horizontal line for significance threshold
plt.axvline(3, color='green', linestyle='--', linewidth=1)  # Add a vertical line for fold change threshold
plt.axvline(-3, color='green', linestyle='--', linewidth=1)  # Add a vertical line for fold change threshold
output_local = '/content/drive/MyDrive/20440_Project/Figures/'
plt.savefig(output_local + 'VolcanoPlot_Terminal_Ileum', format = 'jpeg' , bbox_inches='tight')

# Filter only columns with p-values less than 0.05
significant_genes = EX_FC_pvals.loc[:, EX_FC_pvals.iloc[-1] < 0.05]

# Filter only columns with absolute FC values exceeding 3
significant_genes2 = significant_genes.loc[:, (abs(significant_genes.iloc[-2]) > 3)]

# Iterate over columns in the DataFrame
for column in significant_genes2.columns:
    # Check if the absolute FC value is within the desired range
    if abs(significant_genes2[column].iloc[-2]) <= 3:
        # Drop the column if the condition is not met
        significant_genes2.drop(column, axis=1, inplace=True)

GOI_terminal_ileum = significant_genes2


In [ ]:
# Define the index names for healthy and disease samples
Healthy = 'Normal'
Disease = 'UC'

# Selecting rows
Healthy_rows = ascending_colon_data[ascending_colon_data.index == Healthy]
UC_rows = ascending_colon_data[ascending_colon_data.index == Disease]

# Initialize an empty DataFrame to store p-values
p_values_df = pd.DataFrame(columns=ascending_colon_data.columns)
fold_change_df = pd.DataFrame(columns = ascending_colon_data.columns )

# Calculate average fold change (FC) for each gene
for column in ascending_colon_data.columns:
    # Convert data to numeric to handle missing values
    Healthy_column_numeric = pd.to_numeric(Healthy_rows[column], errors='coerce')
    UC_column_numeric = pd.to_numeric(UC_rows[column], errors='coerce')

    # Drop missing values
    Healthy_column_numeric = Healthy_column_numeric.dropna()
    UC_column_numeric = UC_column_numeric.dropna()

    # Calculate mean expression in healthy and disease samples
    mean_expression_healthy = Healthy_column_numeric.mean()
    mean_expression_UC = UC_column_numeric.mean()

    # Calculate fold change
    fold_change = mean_expression_UC / mean_expression_healthy
    fold_change_df[column] = [fold_change]

    # Perform t-test if both healthy and disease samples exist
    if not (Healthy_column_numeric.empty or UC_column_numeric.empty):
        t_stat, p_val = ttest_ind(UC_column_numeric, Healthy_column_numeric)
        # Add p-value for each gene to the DataFrame
        p_values_df[column] = [p_val]

# Set index names for fold change and p-values dataframes
fold_change_df.index = ['Fold change']
p_values_df.index = ['p-val']

# Concatenate the DataFrame containing FC and p-values to the original DataFrame
EX_FC_pvals = pd.concat([ascending_colon_data, fold_change_df, p_values_df], ignore_index = False)

#WORKS
#Transforming all gene p-vals and FC
p_vals = EX_FC_pvals.iloc[-1,:]
Fold_changes = EX_FC_pvals.iloc[-2,:]

# Convert p-values to numeric (in case they are not already)
p_vals = pd.to_numeric(p_vals, errors='coerce')
# Take the logarithm of p-values (base 10) for better visualization
log10pval_all = -np.log10(p_vals)

# Convert p-values to numeric (in case they are not already)
Fold_changes = pd.to_numeric(Fold_changes, errors='coerce')
# Take the logarithm of p-values (base 10) for better visualization
log2FC_all = -np.log2(Fold_changes)

# Plotting volcano plot

VolcanoPlot = plt.scatter(log2FC_all, log10pval_all, alpha = 0.5, color = 'blue', label = 'all' )
plt.xlabel("log2(Fold Change) [UC/Healthy]")
plt.ylabel("-log(p-value)")
plt.title("Volcano plot differentially expressed genes between UC and Healthy in ascending colon")
plt.axhline(-np.log10(0.05), color='black', linestyle='--', linewidth=1)  # Add a horizontal line for significance threshold
plt.axvline(3, color='green', linestyle='--', linewidth=1)  # Add a vertical line for fold change threshold
plt.axvline(-3, color='green', linestyle='--', linewidth=1)  # Add a vertical line for fold change threshold
output_local = '/content/drive/MyDrive/20440_Project/Figures/'
plt.savefig(output_local + 'VolcanoPlot_Ascending_Colon', format = 'jpeg' , bbox_inches='tight')

# Filter only columns with p-values less than 0.05
significant_genes = EX_FC_pvals.loc[:, EX_FC_pvals.iloc[-1] < 0.05]

# Filter only columns with absolute FC values exceeding 3
significant_genes2 = significant_genes.loc[:, (abs(significant_genes.iloc[-2]) > 3)]

# Iterate over columns in the DataFrame
for column in significant_genes2.columns:
    # Check if the absolute FC value is within the desired range
    if abs(significant_genes2[column].iloc[-2]) <= 3:
        # Drop the column if the condition is not met
        significant_genes2.drop(column, axis=1, inplace=True)

GOI_ascending_colon = significant_genes2


In [ ]:
#Print only the last two rows
GOI_sigmoid_colon2 = GOI_sigmoid_colon.transpose()
GOI_sigmoid_colon3 = GOI_sigmoid_colon2.iloc[:,[-2,-1]] #gene, FC, p-val

GOI_descending_colon2 = GOI_descending_colon.transpose()
GOI_descending_colon3 = GOI_descending_colon2.iloc[:,[-2,-1]] #gene, FC, p-val

GOI_terminal_ileum2 = GOI_terminal_ileum.transpose()
GOI_terminal_ileum3 = GOI_terminal_ileum2.iloc[:,[-2,-1]] #gene, FC, p-val

GOI_ascending_colon2 = GOI_ascending_colon.transpose()
GOI_ascending_colon3 = GOI_ascending_colon2.iloc[:,[-2,-1]] #gene, FC, p-val



In [ ]:
IDS = merged_df_trans2.transpose()
IDS = IDS.iloc[:, [0,1]]

# Adding NM_ID and GeneID to GOI for each
GOI_sigmoid_colon_final    = GOI_sigmoid_colon3.merge(   IDS, left_index=True, right_index=True)
GOI_descending_colon_final = GOI_descending_colon3.merge(IDS, left_index=True, right_index=True)
GOI_terminal_ileum_final   = GOI_terminal_ileum3.merge(  IDS, left_index=True, right_index=True)
GOI_ascending_colon_final  = GOI_ascending_colon3.merge( IDS, left_index=True, right_index=True)

In [ ]:
#Extractning GeneIDs from dataframes
print("sigmoid colon [=] SC")
print("descending colon [=] DC")
print("terminal ileum [=] TI")
print("ascending ileum [=] AC")
GOI_SC_GeneNames = GOI_sigmoid_colon_final.iloc[:, 3]
GOI_DC_GeneNames = GOI_descending_colon_final.iloc[:, 3]
GOI_TI_GeneNames = GOI_terminal_ileum_final.iloc[:, 3]
GOI_AC_GeneNames = GOI_ascending_colon_final.iloc[:, 3]

# Assuming df1, df2, df3, and df4 are your DataFrames containing GeneIDs

# Extract unique GeneIDs from each DataFrame
gene_ids_df1 = set(GOI_SC_GeneNames)
gene_ids_df2 = set(GOI_DC_GeneNames)
gene_ids_df3 = set(GOI_TI_GeneNames)
gene_ids_df4 = set(GOI_AC_GeneNames)

# Determine the overlap between the sets of GeneIDs
overlap_df1_df2 = gene_ids_df1.intersection(gene_ids_df2)
overlap_df1_df3 = gene_ids_df1.intersection(gene_ids_df3)
overlap_df1_df4 = gene_ids_df1.intersection(gene_ids_df4)
overlap_df2_df3 = gene_ids_df2.intersection(gene_ids_df3)
overlap_df2_df4 = gene_ids_df2.intersection(gene_ids_df4)
overlap_df3_df4 = gene_ids_df3.intersection(gene_ids_df4)

# Print the number of overlapping GeneIDs in each pair of DataFrames
print("Overlap between SC and DC:", len(overlap_df1_df2))
print("Overlap between SC and TI:", len(overlap_df1_df3))
print("Overlap between SC and AC:", len(overlap_df1_df4))
print("Overlap between DC and TI:", len(overlap_df2_df3))
print("Overlap between DC and AC:", len(overlap_df2_df4))
print("Overlap between TI and AC:", len(overlap_df3_df4))

# Compute overlaps between all combinations of three sets
overlap_1_2_3 = gene_ids_df1.intersection(gene_ids_df2, gene_ids_df3)
overlap_1_2_4 = gene_ids_df1.intersection(gene_ids_df2, gene_ids_df4)
overlap_1_3_4 = gene_ids_df1.intersection(gene_ids_df3, gene_ids_df4)
overlap_2_3_4 = gene_ids_df2.intersection(gene_ids_df3, gene_ids_df4)

# Print the sizes of the overlaps
print("Overlap between DataFrames SC, DC, and TI:", len(overlap_1_2_3))
print("Overlap between DataFrames SC, DC, and AC:", len(overlap_1_2_4))
print("Overlap between DataFrames SC, TI, and AC:", len(overlap_1_3_4))
print("Overlap between DataFrames DC, TI, and AC:", len(overlap_2_3_4))

# Compute the total overlap between all four DataFrames
total_overlap = gene_ids_df1.intersection(gene_ids_df2, gene_ids_df3, gene_ids_df4)
print("Total overlap between all four DataFrames:", len(total_overlap))

In [ ]:
print("Significanlty significantly expressed genes between sigmoid, descending, and ascending colon between UC and Healthy", overlap_1_2_4)
print("Significanlty significantly expressed genes between sigmoid colon, descending colon , and terminal ileum", overlap_1_2_3)

# Random Forest


In [ ]:
#print(sample_info)
# Assuming df is your DataFrame and 'column_name' is the name of the column
values_to_find = ['HCLS1', 'CDH24', 'CA314451', 'DCUN1D1', 'POLK', 'BAIAP3']

# Filter the DataFrame based on the condition
filtered_df = merged_df_reorg[merged_df_reorg['GeneID'].isin(values_to_find)]

# Print the rows where the values are found
if not filtered_df.empty:
    print("Rows where the values are found:")
    print(filtered_df)
else:
    print("None of the specified values are found in the column")


In [ ]:
P = filtered_df.index
Info_GOI = merged_df_reorg.iloc[P, :]
Info_GOI = Info_GOI.iloc[1:, [1] + list(range(3, len(Info_GOI.columns)))]
Info_GOI = Info_GOI.transpose()
Info_GOI_trans = Info_GOI.copy()  # Make a copy of the transposed DataFrame
Info_GOI_trans.columns = ['POLKa', 'DCUN1D1a', 'CA314451', 'CDH24a', 'CDH24b', 'DCUN1D1b', 'POLKb', 'POLKc', 'DCUN1D1c', 'BAIAP3', 'POLKd']
Info_GOI_trans = Info_GOI_trans.iloc[1:,].astype(float)
print(Info_GOI_trans.info())

#Feature names
Feature_names = Info_GOI_trans.columns

Info_GOI_trans['Diagnosis'] = Info_GOI_trans.index.str.split().str[0]

# Encoding target variable
label_encoder = LabelEncoder()
Diag_encoded = label_encoder.fit_transform(Info_GOI_trans['Diagnosis'])

# Dropping the original diagnosis column
Info_GOI_trans = Info_GOI_trans.drop(columns=['Diagnosis'])

# Assign features and target variable
Features_encoded = pd.get_dummies(Info_GOI_trans)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(Features_encoded, Diag_encoded, test_size=0.2, random_state=42)

# Initialize and train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Making predictions on the testing set
y_pred = rf_classifier.predict(X_test)

# Evaluating the classifier's performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

from sklearn.metrics import confusion_matrix

# Assuming y_true are the true labels and y_pred are the predicted labels
cm = confusion_matrix(y_test, y_pred)
TN, FP, FN, TP = cm.ravel()

sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

print("Sensitivity:", sensitivity)
print("Specificity:", specificity)


In [ ]:
# Initialize the figure and axis

plt.figure(figsize=(20, 20), dpi=300)
# Plot the decision tree
Tree = plot_tree(rf_classifier.estimators_[0],
          feature_names=Features_encoded.columns,
          class_names=label_encoder.classes_,
          filled=True,
          rounded=True,
          fontsize=9,  # Adjust font size
          node_ids=True,  # Add node IDs for better reference
          proportion=False,  # Adjusts leaf node sizes based on sample proportion
          impurity=True,  # Don't display impurity at each node
          precision=2)  # Adjust the precision of displayed values


# Add title and labels
plt.title("Decision Tree Visualization", fontsize=30)
plt.xlabel("Features", fontsize=12)
plt.ylabel("Classes", fontsize=12)

# Remove box around the plot
plt.box(False)

# Show plot
output_local = '/content/drive/MyDrive/20440_Project/Figures/'
plt.savefig(output_local + 'RF_Tree', format = 'jpeg' , bbox_inches='tight')

In [ ]:
#Continue with POLKb and BAIAP3 for diagnosis since gini score is the highest
P = filtered_df.index
Info_GOI = merged_df_reorg.iloc[P, :]
Info_GOI = Info_GOI.iloc[1:, [1] + list(range(3, len(Info_GOI.columns)))]
Info_GOI = Info_GOI.transpose()
Info_GOI_trans = Info_GOI.copy()  # Make a copy of the transposed DataFrame
Info_GOI_trans.columns = ['POLKa', 'DCUN1D1a', 'CA314451', 'CDH24a', 'CDH24b', 'DCUN1D1b', 'POLKb', 'POLKc', 'DCUN1D1c', 'BAIAP3', 'POLKd']
Info_GOI_trans = Info_GOI_trans.drop(columns=['POLKa', 'DCUN1D1a', 'CA314451', 'CDH24a', 'CDH24b', 'DCUN1D1b', 'POLKc', 'DCUN1D1c', 'POLKd'])
Info_GOI_trans = Info_GOI_trans.iloc[1:,].astype(float)
print(Info_GOI_trans.info())

#Feature names
Feature_names = Info_GOI_trans.columns

Info_GOI_trans['Diagnosis'] = Info_GOI_trans.index.str.split().str[0]

# Encoding target variable
label_encoder = LabelEncoder()
Diag_encoded = label_encoder.fit_transform(Info_GOI_trans['Diagnosis'])

# Dropping the original diagnosis column
Info_GOI_trans = Info_GOI_trans.drop(columns=['Diagnosis'])

# Assign features and target variable
Features_encoded = pd.get_dummies(Info_GOI_trans)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(Features_encoded, Diag_encoded, test_size=0.2, random_state=42)

# Initialize and train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Making predictions on the testing set
y_pred = rf_classifier.predict(X_test)

# Evaluating the classifier's performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

from sklearn.metrics import confusion_matrix

# Assuming y_true are the true labels and y_pred are the predicted labels
cm = confusion_matrix(y_test, y_pred)
TN, FP, FN, TP = cm.ravel()

sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

# Initialize the figure and axis

plt.figure(figsize=(20, 20), dpi=300)
# Plot the decision tree
Tree2 = plot_tree(rf_classifier.estimators_[0],
          feature_names=Features_encoded.columns,
          class_names=label_encoder.classes_,
          filled=True,
          rounded=True,
          fontsize=8,  # Adjust font size
          node_ids=True,  # Add node IDs for better reference
          proportion=False,  # Adjusts leaf node sizes based on sample proportion
          impurity=True,  # Don't display impurity at each node
          precision=2)  # Adjust the precision of displayed values


# Add title and labels
plt.title("Decision Tree Visualization", fontsize=30)
plt.xlabel("Features", fontsize=12)
plt.ylabel("Classes", fontsize=12)

# Remove box around the plot
plt.box(False)

# Show plot
output_local = '/content/drive/MyDrive/20440_Project/Figures/'
plt.savefig(output_local + 'Tree_2Genes', format = 'jpeg' , bbox_inches='tight')

In [ ]:
#Determining if there is co-expression of the two genes across all samples UC and Healthy
P = filtered_df.index
Info_GOI = merged_df_reorg.iloc[P, :]
Info_GOI = Info_GOI.iloc[1:, [1] + list(range(3, len(Info_GOI.columns)))]
Info_GOI = Info_GOI.transpose()
Info_GOI_trans = Info_GOI.copy()  # Make a copy of the transposed DataFrame
Info_GOI_trans.columns = ['POLKa', 'DCUN1D1a', 'CA314451', 'CDH24a', 'CDH24b', 'DCUN1D1b', 'POLKb', 'POLKc', 'DCUN1D1c', 'BAIAP3', 'POLKd']
Info_GOI_trans = Info_GOI_trans.drop(columns=['POLKa', 'DCUN1D1a', 'CA314451', 'CDH24a', 'CDH24b', 'DCUN1D1b', 'POLKc', 'DCUN1D1c', 'POLKd'])
Info_GOI_trans = Info_GOI_trans.iloc[1:,].astype(float)
Info_GOI_trans['Diagnosis'] = Info_GOI_trans.index.str.split().str[0]
Info_GOI_trans = Info_GOI_trans.set_index('Diagnosis')

# Selecting rows
Healthy_rows = Info_GOI_trans[Info_GOI_trans.index == 'Normal']
UC_rows = Info_GOI_trans[Info_GOI_trans.index == 'UC']

#Healthy
POLK_H = Healthy_rows.iloc[:,0]
BAIAP3_H = Healthy_rows.iloc[:,1]
#UC
POLK_UC = UC_rows.iloc[:,0]
BAIAP3_UC = UC_rows.iloc[:,1]

plt.plot(POLK_H, BAIAP3_H, 'o', color = 'blue', label = 'Healthy')
plt.plot(POLK_UC, BAIAP3_UC, 'o', color = 'red', label = 'UC')
plt.xlabel('POLK Expression')
plt.ylabel('BAIAP3 Expression')
plt.legend()
plt.title("Co-occurance of Differential BAIAP3 and POLK Expression")
output_local = '/content/drive/MyDrive/20440_Project/Figures/'
plt.savefig(output_local + 'POLK_BAIAP3_Expressions', format = 'jpeg' , bbox_inches='tight')

# Misc

## Decision Tree for 1-2 Genes and Patient History


In [ ]:
sample_info

In [ ]:
sample_info
Features_new = ['Birth.Date', 'Gender', 'Ethnicity', 'Diagnosis.Date', 'Family.History', 'Smoking.Status']
PT_history = sample_info[Features_new]

#Changing birthdate to age
PT_history['Birth.Year'] = PT_history['Birth.Date'].str.split('/').str[-1]
PT_history['Birth.Year'] = pd.to_numeric(PT_history['Birth.Year']) + 1900
PT_history['Age'] = 2008-PT_history['Birth.Year']
PT_history.drop(columns = ['Birth.Year', 'Birth.Date'], inplace = True)

#Changing diagnosis date to years
PT_history['DiagnosisYear'] = PT_history['Diagnosis.Date'].str.split('/').str[-1]
PT_history['Time_of_UC'] = 2008-pd.to_numeric(PT_history['DiagnosisYear'])
PT_history.drop(columns = ['DiagnosisYear', 'Diagnosis.Date'], inplace = True)

# Define smoking status mapping
smoking_mapping = {'NaN': 0, 'never': 1, 'current': 2, 'ex': 3, 'unknown': 4}
PT_history['Smoking.Status'] = PT_history['Smoking.Status'].map(smoking_mapping)

# Convert gender from words (M/F) to numbers
gender_mapping = {'M': 0, 'F': 1}
PT_history['Gender'] = PT_history['Gender'].map(gender_mapping)

# Define ethnicity mapping
ethnicity_mapping = {'Caucasian': 0, 'Jewish': 1, 'Asian': 2}
PT_history['Ethnicity'] = PT_history['Ethnicity'].map(ethnicity_mapping)

#Healthy vs UC
# Create a new column 'Disease_Status' based on the values in 'Time_of_UC'
PT_history['Disease_Status'] = np.where(PT_history['Time_of_UC'].isna(), 'Healthy', 'UC')
PT_history.fillna(0, inplace=True)
PT_history.drop(columns = ['Time_of_UC'], inplace = True)
print(PT_history.head())




In [ ]:
# Define features (X) and target variable (y)
X = PT_history.drop(columns=['Disease_Status'])
y = PT_history['Disease_Status']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the random forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Predict on the testing set
y_pred = rf_classifier.predict(X_test)

# Evaluating the classifier's performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
TN, FP, FN, TP = cm.ravel()

# Compute sensitivity and specificity
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

# Visualize a decision tree from the random forest (assuming plot_tree function is available)
plt.figure(figsize=(20, 20), dpi=300)
Tree = plot_tree(rf_classifier.estimators_[0],
          feature_names=X.columns,
          class_names=rf_classifier.classes_,
          filled=True,
          rounded=True,
          fontsize=12,
          node_ids=True,
          proportion=False,
          impurity=True,
          precision=2)

plt.title("Decision Tree Visualization", fontsize=30)
plt.xlabel("Features", fontsize=12)
plt.ylabel("Classes", fontsize=12)
plt.box(False)

# Save the plot
output_local = '/content/drive/MyDrive/20440_Project/Figures/'
plt.savefig(output_local + 'RF_Tree3', format='jpeg', bbox_inches='tight')

In [ ]:
#uploading GEO accesion data
file_path = '/content/drive/MyDrive/20440_Project/GSE11223_series_matrix.txt'
#converting txt to data frame (removing metadata)
raw_file2 = pd.read_csv(file_path, delimiter='\t',skiprows = 139)

In [ ]:
#Renaming ID_REF column on raw data
column = list(raw_file2.columns)
raw_file2.rename(columns= {column[0]: 'ID_REF'}, inplace = True)
#making sure everything is same dtype before merge
raw_file2.loc[:, 'ID_REF'] = raw_file2.loc[:, 'ID_REF'].astype(str)
#replacing NaN values with 0
raw_file2 = raw_file2.fillna(0)

#adding NM_... ID and Gene Names to patient values
merged_df = pd.merge(raw_file2, ID_abb, on='ID_REF', how ='left')
#extracting the name of the columns
column_names = list(merged_df.columns)

#adding NM_ID, GeneID to front of df
new_order = column_names[-2:] + column_names[:-2]
merged_df_reorg = merged_df[new_order]
merged_df_reorg

In [ ]:
sample_info

In [ ]:
#print(sample_info)
# Assuming df is your DataFrame and 'column_name' is the name of the column
values_to_find = ['POLK', 'BAIAP3']

# Filter the DataFrame based on the condition
filtered_df = merged_df_reorg[merged_df_reorg['GeneID'].isin(values_to_find)]

new_columns = ["POLK", 'POLK', 'POLK', 'BAIAP3', 'POLK']
New_filtered = filtered_df.transpose()
New_filtered.columns = new_columns

sample_info
Features_new2 = ['GSM','Birth.Date', 'Gender', 'Ethnicity', 'Diagnosis.Date', 'Family.History', 'Smoking.Status']
PT_history2 = sample_info[Features_new2]

#merge GSM column from sample_info and all of PT_history2 on it's index which is GSM values

# Merge GSM column from sample_info with PT_history2 on index
PT_Final= pd.merge(New_filtered, PT_history2, left_index=True, right_on='GSM')

#Changing birthdate to age
PT_Final['Birth.Year'] = PT_Final['Birth.Date'].str.split('/').str[-1]
PT_Final['Birth.Year'] = pd.to_numeric(PT_Final['Birth.Year']) + 1900
PT_Final['Age'] = 2008-PT_Final['Birth.Year']
PT_Final.drop(columns = ['Birth.Year', 'Birth.Date', 'GSM'], inplace = True)

#Changing diagnosis date to years
PT_Final['DiagnosisYear'] = PT_Final['Diagnosis.Date'].str.split('/').str[-1]
PT_Final['Time_of_UC'] = 2008-pd.to_numeric(PT_Final['DiagnosisYear'])
PT_Final.drop(columns = ['DiagnosisYear', 'Diagnosis.Date'], inplace = True)

# Define smoking status mapping
smoking_mapping = {'NaN': 0, 'never': 1, 'current': 2, 'ex': 3, 'unknown': 4}
PT_Final['Smoking.Status'] = PT_Final['Smoking.Status'].map(smoking_mapping)

# Convert gender from words (M/F) to numbers
gender_mapping = {'M': 0, 'F': 1}
PT_Final['Gender'] = PT_Final['Gender'].map(gender_mapping)

# Define ethnicity mapping
ethnicity_mapping = {'Caucasian': 0, 'Jewish': 1, 'Asian': 2}
PT_Final['Ethnicity'] = PT_Final['Ethnicity'].map(ethnicity_mapping)

#Healthy vs UC
# Create a new column 'Disease_Status' based on the values in 'Time_of_UC'
PT_Final['Disease_Status'] = np.where(PT_Final['Time_of_UC'].isna(), 'Healthy', 'UC')
PT_Final.fillna(0, inplace=True)
PT_Final.drop(columns = ['Time_of_UC'], inplace = True)

# Define features (X) and target variable (y)
X = PT_Final.drop(columns=['Disease_Status'])
y = PT_Final['Disease_Status']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the random forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Predict on the testing set
y_pred = rf_classifier.predict(X_test)

# Evaluating the classifier's performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
TN, FP, FN, TP = cm.ravel()

# Compute sensitivity and specificity
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

# Visualize a decision tree from the random forest (assuming plot_tree function is available)
plt.figure(figsize=(20, 20), dpi=300)
Tree = plot_tree(rf_classifier.estimators_[0],
          feature_names=X.columns,
          class_names=rf_classifier.classes_,
          filled=True,
          rounded=True,
          fontsize=12,
          node_ids=True,
          proportion=False,
          impurity=True,
          precision=2)

plt.title("Decision Tree Visualization", fontsize=30)
plt.xlabel("Features", fontsize=12)
plt.ylabel("Classes", fontsize=12)
plt.box(False)

# Save the plot
output_local = '/content/drive/MyDrive/20440_Project/Figures/'
plt.savefig(output_local + 'RF_Tree4', format='jpeg', bbox_inches='tight')

## Going to perform a differential gene expression analysis by birthyear


In [ ]:
# looking at age distribution of disease status
GSM = sample_info.iloc[:, [0,3,4,7]]

# Split the date strings by '/'
birth_years = GSM.iloc[:,1].str.split('/')

# Select the last element from each split result
birth_years = birth_years.str[-1]

# Convert birth years to numeric
birth_years = pd.to_numeric(birth_years, errors='coerce')
# Add birth_years column to GSM DataFrame
GSM['birth_years'] = birth_years

# Get the current year
current_year = datetime.datetime.now().year

# Assume a reference point for the century (e.g., 1900 for 'YY' format)
century_reference = 1900

# Convert birth years to full years
GSM['birth_years'] = century_reference + GSM['birth_years']

# Calculate age by subtracting birth year from the current year
GSM['age'] = current_year - GSM['birth_years']

# Selecting rows
Healthy_rows_AGE = GSM[GSM.iloc[:,3].isna()]
UC_rows_AGE = GSM[~GSM.iloc[:,3].isna()]

# Sorting the DataFrame in descending order based on age
Healthy_rows_AGE_sorted = Healthy_rows_AGE.sort_values(by=Healthy_rows_AGE.columns[4], ascending=False)
UC_rows_AGE_sorted = UC_rows_AGE.sort_values(by=UC_rows_AGE.columns[4], ascending=False)


AgeHist = plt.hist(Healthy_rows_AGE_sorted.iloc[3:,4], bins = 4, alpha = 0.5, label = "Healthy", edgecolor = 'black')
plt.hist(UC_rows_AGE_sorted.iloc[:,4], bins = 4, alpha = 0.5, label = "UC",edgecolor = 'black')
plt.xlabel("Birth Year")
plt.ylabel("Number of Samples")
plt.title("Age Distribution by Disease Status")
plt.legend()

plt.savefig(output_local + 'AgeHist', format = 'jpeg')

In [ ]:

stat, p = shapiro(Healthy_rows_AGE_sorted.iloc[3:,4])
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('Healthy Samples Not Normally Distributed(fail to reject H0)')
else:
    print('Healthy Samples Normally Distributed (reject H0)')

stat, p = shapiro(UC_rows_AGE_sorted.iloc[:,4])
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('UC Samples Not Normally Distributed(fail to reject H0)')
else:
    print('UC Samples Normally Distributed (reject H0)')

In [ ]:
# Combine age data for healthy and UC groups into a single list
data = [Healthy_rows_AGE_sorted.iloc[3:,4], UC_rows_AGE_sorted.iloc[:,4]]

# Create a boxplot
plt.figure(figsize=(8, 6))
AgeBoxPlot = plt.boxplot(data, labels=['Healthy', 'UC'], patch_artist=True, boxprops=dict(facecolor='lightblue'), medianprops=dict(color='blue'))
plt.xlabel('Disease Status')
plt.ylabel('Birth Year')
plt.title('Age Distribution by Disease Status')
plt.grid(True)  # Add gridlines

# Perform t-test
stat, p = ttest_ind(*data)
alpha = 0.05

# Add asterisks for significant differences
if p < alpha:
    plt.text(1.5, max(max(data[0]), max(data[1])) + 1, '*', ha='center', va='bottom', color='Black', fontsize=30)

# Extend the y-axis
extension_amount = 10  # Adjust this value as needed
plt.ylim(min(min(data[0]), min(data[1])) - extension_amount, max(max(data[0]), max(data[1])) + extension_amount)

# Show the plot
plt.savefig(output_local + 'AgeBoxPlot', format = 'jpeg')


In [ ]:
# Extract birth years and calculate ages
File = raw_file.transpose()

birth_years = pd.to_numeric(File.iloc[:, 3].str.split('/').str[-1], errors='coerce')


File['birth_years'] = century_reference + birth_years

# Calculate ages
File['age'] = current_year - File['birth_years']

# Bin ages
bins = [0, 40, 60, 80, 100]
labels = ['0-40', '41-60', '61-80', '81-100']
File['age_bin'] = pd.cut(File['age'], bins=bins, labels=labels, right=False)
File = File.set_index('age_bin')
#Dataframe that has age bin as index and each row is a different column
File = File.iloc[1:, 89:-3]
# Initialize an empty DataFrame to store p-values
p_values_df = pd.DataFrame(index=labels, columns=File.columns)

# Perform t-test for each gene expression across each age bin
for label in labels:
    age_group = File.loc[File.index == label]
    for column in File.columns:
        # Convert data to numeric to handle missing values
        age_group_numeric = pd.to_numeric(age_group[column], errors='coerce')
        first_age_group_numeric = pd.to_numeric(File.loc[File.index == labels[0], column], errors='coerce')
        # Drop missing values
        age_group_numeric = age_group_numeric.dropna()
        first_age_group_numeric = first_age_group_numeric.dropna()
        # Perform t-test for each age group compared to the first one
        if not (age_group_numeric.empty or first_age_group_numeric.empty):
            _, p_val = ttest_ind(age_group_numeric, first_age_group_numeric)
            p_values_df.at[label, column] = p_val

# Bonferroni correction
p_values_corrected = p_values_df.apply(lambda x: x * len(p_values_df.columns), axis=1)

# Print the DataFrame with corrected p-values
print(p_values_corrected.head())


In [ ]:
from scipy.stats import f_oneway
# Extract birth years and calculate ages
File = raw_file.transpose()

birth_years = pd.to_numeric(File.iloc[:, 3].str.split('/').str[-1], errors='coerce')


File['birth_years'] = century_reference + birth_years

# Calculate ages
File['age'] = current_year - File['birth_years']

# Bin ages
bins = [0, 40, 60, 80, 100]
labels = ['0-40', '41-60', '61-80', '81-100']
File['age_bin'] = pd.cut(File['age'], bins=bins, labels=labels, right=False)
File = File.set_index('age_bin')
#Dataframe that has age bin as index and each row is a different column
File = File.iloc[1:, 89:-3]
# Assuming 'File' contains gene expression data with age groups in columns
# Each column represents a different age group

# Define the age group columns
age_group_columns = ['0-40', '41-60', '61-80', '81-100']

# Initialize an empty DataFrame to store ANOVA results
anova_results = pd.DataFrame(index=File.columns, columns=['F-statistic', 'p-value'])

# Perform ANOVA test for each gene
for column in File.columns:
    # Extract expression values for each age group
    age_group_data = [File[column][File.index == age_group].dropna() for age_group in age_group_columns]

    # Perform ANOVA test
    f_statistic, p_value = f_oneway(*age_group_data)

    # Store results in the DataFrame
    anova_results.loc[column, 'F-statistic'] = f_statistic
    anova_results.loc[column, 'p-value'] = p_value

# Print ANOVA results
print(anova_results.head())


In [ ]:
# Remove the '.number' suffix from the index
Sample_type_ex.index = Sample_type_ex.index.str.replace(r'\.\d+$', '', regex=True)
split = Sample_type_ex.index.str.split()
first_word = split.str[0]
second_word = split.str[1]
third_word = split.str[2]
fourth_word = split.str[3]
fourth = fourth_word.str.split('.')
Sample_type_ex['Condition'] = first_word
Sample_type_ex['Biopsy'] = third_word + fourth_word
print(Sample_type_ex.info)